In [1]:
import torch.nn.functional as F
import torch.nn as nn

from datasets import load_dataset

In [2]:
dataset = load_dataset("cfilt/iitb-english-hindi")

Found cached dataset parquet (/home/administrator/.cache/huggingface/datasets/cfilt___parquet/cfilt--iitb-english-hindi-911387c6837f8b91/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset['train']['translation'][:10]

[{'en': 'Give your application an accessibility workout',
  'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'},
 {'en': 'Accerciser Accessibility Explorer',
  'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'},
 {'en': 'The default plugin layout for the bottom panel',
  'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'The default plugin layout for the top panel',
  'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'},
 {'en': 'A list of plugins that are disabled by default',
  'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'},
 {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'},
 {'en': 'The duration of the highlight box when selecting accessible nodes',
  'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'},
 {'en': 'Highlight border color',
  'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'},
 {'en': 'The color and opacity of the highlight border.',
  'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '},
 {'en': 'Highlight fill color', 'hi': 

# Tokenizer

In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
def training_corpus(dtype='train', lang='hi'):
    l_dataset = len(dataset[dtype])
    for i in range(0, l_dataset, 1000):
        yield [dataset[dtype][i + j]["translation"][lang] for j in range(min(1000,l_dataset-i))]


In [ ]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
for hi_data in training_corpus(dtype='train',lang='hi'):
    break;
for en_data in training_corpus(dtype='train',lang='en'):
    break;

In [ ]:
tokens = old_tokenizer.tokenize(d[0])
len(tokens),tokens

In [ ]:
EN_VOCAB_SIZE = 75000
HI_VOCAB_SIZE = 75000

In [ ]:
hi_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus('train'), HI_VOCAB_SIZE)
en_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus('train', lang='en'), EN_VOCAB_SIZE)

In [ ]:
en_tokenizer.save_pretrained("eng-tokenizer")

In [ ]:
hi_tokenizer.save_pretrained("hindi-tokenizer")

In [ ]:
tokens = hi_tokenizer.tokenize(hi_data[2])
print(len(tokens),tokens)
hi_tokenizer.convert_tokens_to_string(tokens)

In [ ]:
tokens = en_tokenizer.tokenize(en_data[2])
print(len(tokens),tokens)
en_tokenizer.convert_tokens_to_string(tokens)

### Load Tokenizers from saved

In [4]:
from transformers import AutoTokenizer

hi_tokenizer = AutoTokenizer.from_pretrained("hindi-tokenizer")
en_tokenizer = AutoTokenizer.from_pretrained("eng-tokenizer")

In [5]:
hi_tokenizer.add_special_tokens({'pad_token': '[PAD]', 'cls_token': '<cls>', 'eos_token':'<eos>', 'bos_token' : '<s>'})

en_tokenizer.add_special_tokens({'pad_token': '[PAD]', 'cls_token': '<cls>', 'eos_token':'<eos>', 'bos_token' : '<s>'})

3

In [6]:
from tokenizers.processors import TemplateProcessing
en_tokenizer._tokenizer.post_processor = TemplateProcessing(
    single=en_tokenizer.bos_token + " $A " + en_tokenizer.eos_token,
    special_tokens=[(en_tokenizer.eos_token, en_tokenizer.eos_token_id), (en_tokenizer.bos_token, en_tokenizer.bos_token_id)],
)

In [7]:
en_sen = dataset['train']['translation'][1]['en']

In [8]:
en_tokenizer.encode(en_sen, add_special_tokens = True)

[75003, 60318, 50652, 32916, 75002]

# Translator - Train

In [9]:
import torch

device = "cpu"#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

'cpu'

In [10]:
BS = 4
def collate_data(dataset, hi_tokenizer, en_tokenizer):
    hi_list = []
    en_list = []
    for i in range(BS):
        hi_list.append(dataset['translation'][i]['hi'])
        en_list.append(dataset['translation'][i]['en'])
    yield ( hi_tokenizer(hi_list, padding=True, truncation=True, return_tensors="pt"), 
           en_tokenizer(en_list, padding=True, truncation=True, return_tensors="pt"))

In [11]:
for d in collate_data(dataset['train'], hi_tokenizer, en_tokenizer):
    break;

In [13]:
print(list(map(hi_tokenizer.decode, d[0]['input_ids'])))
print(list(map(en_tokenizer.decode, d[1]['input_ids'])))

2023-07-06 09:52:23.883957: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


['अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें', 'एक्सेर्साइसर पहुंचनीयता अन्वेषक[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]', 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका[PAD][PAD][PAD][PAD][PAD]', 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका[PAD][PAD][PAD][PAD][PAD][PAD][PAD]']
['<s>Give your application an accessibility workout<eos>[PAD]', '<s>Accerciser Accessibility Explorer<eos>[PAD][PAD][PAD][PAD][PAD]', '<s>The default plugin layout for the bottom panel<eos>', '<s>The default plugin layout for the top panel<eos>']


In [ ]:
# import gc
# with torch.no_grad():
#     torch.cuda.empty_cache()
# gc.collect()


In [53]:
# import inspect
# type(model), inspect.getfile(model.forward)

(transformers.models.encoder_decoder.modeling_encoder_decoder.EncoderDecoderModel,
 '/Data/data/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py')

In [ ]:
# %reload_ext autoreload
# %autoreload
# import transformers

In [14]:
criterion = nn.NLLLoss(ignore_index=en_tokenizer.pad_token_id)
# criterion = nn.NLLLoss()


In [15]:
def compute_loss(predictions, targets):
    """Compute our custom loss"""
    predictions = predictions[:, :-1, :].contiguous()
    targets = targets[:, 1:]

    rearranged_output = predictions.view(predictions.shape[0]*predictions.shape[1], -1)
    rearranged_target = targets.contiguous().view(-1)

    loss = criterion(rearranged_output, rearranged_target)

    return loss

In [16]:
import transformers

encoder_config = transformers.BertConfig(vocab_size=len(hi_tokenizer))
decoder_config = transformers.BertConfig(vocab_size = len(en_tokenizer))

config = transformers.EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)
model = transformers.EncoderDecoderModel(config)

In [17]:
model.config.decoder_start_token_id = en_tokenizer.cls_token_id
model.config.pad_token_id = en_tokenizer.pad_token_id
model.config.eos_token_id = en_tokenizer.eos_token_id
model.config.bos_token_id = en_tokenizer.bos_token_id

model = model.to(device)

In [85]:
model.train()

for _d in d:
    for k in _d.keys():
        _d[k].to(device)

optimizer = transformers.AdamW(model.parameters(), lr=1e-4)
epochs = 100

for e in range(epochs):
    optimizer.zero_grad()
    out = model(input_ids=d[0]['input_ids'],
                         attention_mask = d[0]['attention_mask'],
#                          decoder_input_ids = d[1]['input_ids'],
#                          decoder_attention_mask = d[1]['attention_mask'],
                         labels = d[1]['input_ids'])[:2]
    prediction_scores = out[1]
    predictions = F.log_softmax(prediction_scores, dim=-1)
    loss = compute_loss(predictions, d[1]['input_ids'])

    print(f"epoch {e}:", loss.item())
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

/Data/data/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/Data/data/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


epoch 0: 0.005855017341673374
epoch 1: 0.005516148172318935
epoch 2: 0.005290417466312647
epoch 3: 0.005002203397452831
epoch 4: 0.004580722656100988
epoch 5: 0.004125975538045168
epoch 6: 0.003954860847443342
epoch 7: 0.0035962462425231934
epoch 8: 0.003426605137065053
epoch 9: 0.0030861040577292442
epoch 10: 0.002854029880836606
epoch 11: 0.0026409775018692017
epoch 12: 0.002472362946718931
epoch 13: 0.0022445395588874817
epoch 14: 0.002276189625263214
epoch 15: 0.001995246158912778
epoch 16: 0.0018820755649358034
epoch 17: 0.0017808733973652124
epoch 18: 0.0016483981162309647
epoch 19: 0.001557507668621838
epoch 20: 0.001522453618235886
epoch 21: 0.0014177510747686028
epoch 22: 0.0013292607618495822
epoch 23: 0.0012749137822538614
epoch 24: 0.0012189628323540092
epoch 25: 0.0011316483141854405
epoch 26: 0.001088002696633339
epoch 27: 0.0010442814091220498
epoch 28: 0.0010283931624144316
epoch 29: 0.0009499480947852135
epoch 30: 0.0009287231368944049
epoch 31: 0.0008710247348062694
e

### Save Model Weights

In [86]:
PATH = "./translate_hin_to_eng.pth"

In [87]:
torch.save(model.state_dict(), PATH)

### Load Model

In [88]:
import transformers

encoder_config = transformers.BertConfig(vocab_size=len(hi_tokenizer))
decoder_config = transformers.BertConfig(vocab_size = len(en_tokenizer))

config = transformers.EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, decoder_config)
model = transformers.EncoderDecoderModel(config)

model.config.decoder_start_token_id = en_tokenizer.cls_token_id
model.config.pad_token_id = en_tokenizer.pad_token_id
model.config.eos_token_id = en_tokenizer.eos_token_id
model.config.bos_token_id = en_tokenizer.bos_token_id

model.load_state_dict(torch.load(PATH))
model.eval()

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(75004, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

# Evaluate

In [92]:
output = model.generate(input_ids = d[0]['input_ids'])

In [93]:
(list(map(en_tokenizer.decode, d[1]['input_ids'])),"",
list(map(en_tokenizer.decode, output)))

(['<s>Give your application an accessibility workout<eos>[PAD]',
  '<s>Accerciser Accessibility Explorer<eos>[PAD][PAD][PAD][PAD][PAD]',
  '<s>The default plugin layout for the bottom panel<eos>',
  '<s>The default plugin layout for the top panel<eos>'],
 '',
 ['<cls>Give application accessibilityout accessibilityoutout<eos>',
  '<cls>Accerciser Explorer Explorer<eos>[PAD][PAD][PAD][PAD]',
  '<cls>The plugin for bottom for bottom<eos>[PAD]',
  '<cls>The plugin for top<eos>[PAD][PAD][PAD]'])

In [94]:
model.eval()
epoch_loss = 0


optimizer.zero_grad()
out = model(input_ids=d[0]['input_ids'],
                         attention_mask = d[0]['attention_mask'],
#                          decoder_input_ids = d[1]['input_ids'],
#                          decoder_attention_mask = d[1]['attention_mask'],
                         labels = d[1]['input_ids'])

prediction_scores = out.logits
predictions = F.log_softmax(prediction_scores, dim=-1)
loss = compute_loss(predictions, d[1]['input_ids'])
epoch_loss += loss.item()

print("Mean validation loss:", epoch_loss)


Mean validation loss: 0.0001511055597802624


In [95]:
list(map(en_tokenizer.decode, torch.argmax(predictions,dim=-1)))

['Give your application an accessibility workout<eos><eos><eos>',
 'Accerciser Accessibility Explorer<eos><eos> the bottom<eos><eos><eos>',
 'The default plugin layout for the bottom panel<eos><eos>',
 'The default plugin layout for the top panel<eos><eos>']

In [102]:
model.eval()
dec_inp = torch.tensor([[en_tokenizer.cls_token_id],[en_tokenizer.cls_token_id],[en_tokenizer.cls_token_id],[en_tokenizer.cls_token_id]])

for _ in range(8):
    output = model(input_ids = d[0]['input_ids'], decoder_input_ids = dec_inp )

    pred_words = torch.argmax(output.logits, dim=-1)[:,-1:]

    dec_inp = torch.cat((dec_inp,pred_words),dim=1)

In [103]:
list(map(en_tokenizer.decode, dec_inp))

['<cls>Give application accessibilityout accessibilityoutout<eos>',
 '<cls>Accerciser Explorer Explorer<eos> for bottom<eos><eos>',
 '<cls>The plugin for bottom for bottom<eos> panel',
 '<cls>The plugin for top<eos> the panel<eos>']

In [107]:
import inspect
inspect.getfile(model.forward)

'/Data/data/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py'

In [110]:
type(model.decoder)

transformers.models.bert.modeling_bert.BertLMHeadModel

In [109]:
inspect.getfile(model.decoder.forward)

'/Data/data/miniconda3/envs/ml/lib/python3.11/site-packages/transformers/models/bert/modeling_bert.py'

In [106]:
d[1]['input_ids']

tensor([[75003, 10644,   485,  2159,   351, 17137,   681,   541, 75002, 75000],
        [75003, 60318, 50652, 32916, 75002, 75000, 75000, 75000, 75000, 75000],
        [75003,   328,  3546,  5551,  9083,   316,   261,  6706,  7789, 75002],
        [75003,   328,  3546,  5551,  9083,   316,   261,  2307,  7789, 75002]])

In [112]:
sum(p.numel() for p in model.parameters())

315732220